# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

> As a used car dealership, it is important to us to understand what an appropriate market price is for the cars we sell. This dataset contains a large number of used cars with various details about them and how much they sold for. Using this large dataset, we can construct a theoretical model of how much each of these different details contributes to the market value of a car. This model can then, hopefully, be used to predict with decent precision the expected value of other cars that are not in the dataset and have not been sold yet. In doing so, the prices we set can be as attuned as possible to the details of the market, maximizing business profits.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [1]:
import pandas as pd
import numpy as np

In [2]:
vehicles = pd.read_csv('data/vehicles.csv')

In [3]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [4]:
vehicles.condition.unique()

array([nan, 'good', 'excellent', 'fair', 'like new', 'new', 'salvage'],
      dtype=object)

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [5]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [6]:
vehicles_cleaned = vehicles.dropna(subset=['year', 'odometer']).fillna('unknown')

In [7]:
vehicles_cleaned2 = vehicles_cleaned.drop(vehicles_cleaned.query('price >= 400_000 or odometer > 500_000').index)

In [8]:
unknown_manu = vehicles_cleaned2.query('manufacturer == "unknown"')
individual_terms_in_model = pd.Series([item.lower() for sublist in unknown_manu['model'].str.split(' ').to_list() for item in sublist]).value_counts()

In [9]:
individual_terms_in_model.head(20)

scion            1894
4d               1159
hatchback        1007
international     968
freightliner      827
genesis           718
isuzu             653
im                644
sedan             622
hummer            522
g70               437
all               409
oldsmobile        409
npr               391
maserati          375
suzuki            329
saab              328
4300              306
tc                301
xb                293
Name: count, dtype: int64

In [10]:
mapping_dict = {'scion': 'scion', 'freightliner': 'freightliner', 'genesis': 'genesis', 'isuzu': 'isuzu', 'oldsmobile': 'oldsmobile',
                'maserati': 'maserati', 'suzuki': 'suzuki', 'saab': 'saab', 'corvette': 'corvette', 'chevorlet': 'chevrolet',
                'volkswagon': 'volkswagen', 'lamborghini': 'lamborghini', 'rolls royce': 'rolls royce'}

In [11]:
vehicles_cleaned3 = vehicles_cleaned2

for key, value in mapping_dict.items():
    vehicles_cleaned3.loc[vehicles_cleaned3[vehicles_cleaned3['model'].str.contains(key, case=False)].index, 'manufacturer'] = value



In [12]:
vehicles_cleaned3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 419892 entries, 27 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            419892 non-null  int64  
 1   region        419892 non-null  object 
 2   price         419892 non-null  int64  
 3   year          419892 non-null  float64
 4   manufacturer  419892 non-null  object 
 5   model         419892 non-null  object 
 6   condition     419892 non-null  object 
 7   cylinders     419892 non-null  object 
 8   fuel          419892 non-null  object 
 9   odometer      419892 non-null  float64
 10  title_status  419892 non-null  object 
 11  transmission  419892 non-null  object 
 12  VIN           419892 non-null  object 
 13  drive         419892 non-null  object 
 14  size          419892 non-null  object 
 15  type          419892 non-null  object 
 16  paint_color   419892 non-null  object 
 17  state         419892 non-null  object 
dtypes: float

In [13]:
vehicles_cleaned4 = vehicles_cleaned3.drop(index=vehicles_cleaned3.query('manufacturer == "unknown"').index)
vehicles_cleaned4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 410083 entries, 27 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            410083 non-null  int64  
 1   region        410083 non-null  object 
 2   price         410083 non-null  int64  
 3   year          410083 non-null  float64
 4   manufacturer  410083 non-null  object 
 5   model         410083 non-null  object 
 6   condition     410083 non-null  object 
 7   cylinders     410083 non-null  object 
 8   fuel          410083 non-null  object 
 9   odometer      410083 non-null  float64
 10  title_status  410083 non-null  object 
 11  transmission  410083 non-null  object 
 12  VIN           410083 non-null  object 
 13  drive         410083 non-null  object 
 14  size          410083 non-null  object 
 15  type          410083 non-null  object 
 16  paint_color   410083 non-null  object 
 17  state         410083 non-null  object 
dtypes: float

In [14]:
vehicles_cleaned5 = vehicles_cleaned4.drop(columns=['id', 'region', 'model', 'VIN', 'state'])
vehicles_cleaned5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 410083 entries, 27 to 426879
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         410083 non-null  int64  
 1   year          410083 non-null  float64
 2   manufacturer  410083 non-null  object 
 3   condition     410083 non-null  object 
 4   cylinders     410083 non-null  object 
 5   fuel          410083 non-null  object 
 6   odometer      410083 non-null  float64
 7   title_status  410083 non-null  object 
 8   transmission  410083 non-null  object 
 9   drive         410083 non-null  object 
 10  size          410083 non-null  object 
 11  type          410083 non-null  object 
 12  paint_color   410083 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 43.8+ MB


In [15]:
current_year = max(vehicles_cleaned5['year'])
vehicles_cleaned6 = vehicles_cleaned5.rename(columns={'year':'age'})
vehicles_cleaned6['age'] = current_year - vehicles_cleaned5['year']

In [16]:
vehicles_cleaned6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 410083 entries, 27 to 426879
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         410083 non-null  int64  
 1   age           410083 non-null  float64
 2   manufacturer  410083 non-null  object 
 3   condition     410083 non-null  object 
 4   cylinders     410083 non-null  object 
 5   fuel          410083 non-null  object 
 6   odometer      410083 non-null  float64
 7   title_status  410083 non-null  object 
 8   transmission  410083 non-null  object 
 9   drive         410083 non-null  object 
 10  size          410083 non-null  object 
 11  type          410083 non-null  object 
 12  paint_color   410083 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 43.8+ MB


In [17]:
from sklearn.preprocessing import PolynomialFeatures

In [31]:
original_numeric_column_names = ['age', 'odometer']
columns_to_poly_transform = vehicles_cleaned6[original_numeric_column_names]
poly_features = PolynomialFeatures(degree=3, include_bias=False).fit(columns_to_poly_transform)
poly_transformed_column_names = list(poly_features.get_feature_names_out())
transformed = pd.DataFrame(poly_features.transform(columns_to_poly_transform), index=vehicles_cleaned6.index, columns=poly_transformed_column_names)

In [32]:
vehicles_cleaned6

,price,age,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
27,33590,8.0,gmc,good,8 cylinders,gas,57923.0,clean,other,unknown,unknown,pickup,white
28,22590,12.0,chevrolet,good,8 cylinders,gas,71229.0,clean,other,unknown,unknown,pickup,blue
29,39590,2.0,chevrolet,good,8 cylinders,gas,19160.0,clean,other,unknown,unknown,pickup,red
30,30990,5.0,toyota,good,8 cylinders,gas,41124.0,clean,other,unknown,unknown,pickup,red
31,15000,9.0,ford,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,3.0,nissan,good,6 cylinders,gas,32226.0,clean,other,fwd,unknown,sedan,unknown
426876,30590,2.0,volvo,good,unknown,gas,12029.0,clean,other,fwd,unknown,sedan,red
426877,34990,2.0,cadillac,good,unknown,diesel,4174.0,clean,other,unknown,unknown,hatchback,white
426878,28990,4.0,lexus,good,6 cylinders,gas,30112.0,clean,other,fwd,unknown,sedan,silver


In [33]:
transformed

,age,odometer,age^2,age odometer,odometer^2,age^3,age^2 odometer,age odometer^2,odometer^3
27,8.0,57923.0,64.0,463384.0,3.355074e+09,512.0,3707072.0,2.684059e+10,1.943359e+14
28,12.0,71229.0,144.0,854748.0,5.073570e+09,1728.0,10256976.0,6.088285e+10,3.613853e+14
29,2.0,19160.0,4.0,38320.0,3.671056e+08,8.0,76640.0,7.342112e+08,7.033743e+12
30,5.0,41124.0,25.0,205620.0,1.691183e+09,125.0,1028100.0,8.455917e+09,6.954823e+13
31,9.0,128000.0,81.0,1152000.0,1.638400e+10,729.0,10368000.0,1.474560e+11,2.097152e+15
...,...,...,...,...,...,...,...,...,...
426875,3.0,32226.0,9.0,96678.0,1.038515e+09,27.0,290034.0,3.115545e+09,3.346719e+13
426876,2.0,12029.0,4.0,24058.0,1.446968e+08,8.0,48116.0,2.893937e+08,1.740558e+12
426877,2.0,4174.0,4.0,8348.0,1.742228e+07,8.0,16696.0,3.484455e+07,7.272058e+10
426878,4.0,30112.0,16.0,120448.0,9.067325e+08,64.0,481792.0,3.626930e+09,2.730353e+13


In [34]:
vehicles_cleaned7 = pd.concat([vehicles_cleaned6, transformed.drop(columns=original_numeric_column_names)], axis='columns')
vehicles_cleaned7

,price,age,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,age^2,age odometer,odometer^2,age^3,age^2 odometer,age odometer^2,odometer^3
27,33590,8.0,gmc,good,8 cylinders,gas,57923.0,clean,other,unknown,unknown,pickup,white,64.0,463384.0,3.355074e+09,512.0,3707072.0,2.684059e+10,1.943359e+14
28,22590,12.0,chevrolet,good,8 cylinders,gas,71229.0,clean,other,unknown,unknown,pickup,blue,144.0,854748.0,5.073570e+09,1728.0,10256976.0,6.088285e+10,3.613853e+14
29,39590,2.0,chevrolet,good,8 cylinders,gas,19160.0,clean,other,unknown,unknown,pickup,red,4.0,38320.0,3.671056e+08,8.0,76640.0,7.342112e+08,7.033743e+12
30,30990,5.0,toyota,good,8 cylinders,gas,41124.0,clean,other,unknown,unknown,pickup,red,25.0,205620.0,1.691183e+09,125.0,1028100.0,8.455917e+09,6.954823e+13
31,15000,9.0,ford,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,81.0,1152000.0,1.638400e+10,729.0,10368000.0,1.474560e+11,2.097152e+15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,3.0,nissan,good,6 cylinders,gas,32226.0,clean,other,fwd,unknown,sedan,unknown,9.0,96678.0,1.038515e+09,27.0,290034.0,3.115545e+09,3.346719e+13
426876,30590,2.0,volvo,good,unknown,gas,12029.0,clean,other,fwd,unknown,sedan,red,4.0,24058.0,1.446968e+08,8.0,48116.0,2.893937e+08,1.740558e+12
426877,34990,2.0,cadillac,good,unknown,diesel,4174.0,clean,other,unknown,unknown,hatchback,white,4.0,8348.0,1.742228e+07,8.0,16696.0,3.484455e+07,7.272058e+10
426878,28990,4.0,lexus,good,6 cylinders,gas,30112.0,clean,other,fwd,unknown,sedan,silver,16.0,120448.0,9.067325e+08,64.0,481792.0,3.626930e+09,2.730353e+13


In [35]:
from sklearn.preprocessing import StandardScaler

In [37]:
scaler = StandardScaler()
vehicles_cleaned8 = vehicles_cleaned7
vehicles_cleaned8[poly_columns] = scaler.fit_transform(vehicles_cleaned8[poly_transformed_column_names])

In [38]:
vehicles_cleaned8

,price,age,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,age^2,age odometer,odometer^2,age^3,age^2 odometer,age odometer^2,odometer^3
27,33590,-0.280488,gmc,good,8 cylinders,gas,-0.530080,clean,other,unknown,unknown,pickup,white,-0.237366,-0.534058,-0.578695,-0.146025,-0.389122,-0.498513,-0.430164
28,22590,0.168749,chevrolet,good,8 cylinders,gas,-0.318745,clean,other,unknown,unknown,pickup,blue,-0.086030,-0.230775,-0.467509,-0.114581,-0.229509,-0.381493,-0.390946
29,39590,-0.954344,chevrolet,good,8 cylinders,gas,-1.145743,clean,other,unknown,unknown,pickup,red,-0.350868,-0.863456,-0.772015,-0.159058,-0.477591,-0.588253,-0.474136
30,30990,-0.617416,toyota,good,8 cylinders,gas,-0.796895,clean,other,unknown,unknown,pickup,red,-0.311142,-0.733809,-0.686348,-0.156032,-0.454405,-0.561710,-0.459460
31,15000,-0.168179,ford,excellent,6 cylinders,gas,0.582934,clean,automatic,rwd,full-size,truck,black,-0.205207,-0.000423,0.264272,-0.140414,-0.226803,-0.083899,0.016553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,-0.842034,nissan,good,6 cylinders,gas,-0.938219,clean,other,fwd,unknown,sedan,unknown,-0.341409,-0.818232,-0.728575,-0.158566,-0.472391,-0.580067,-0.467930
426876,30590,-0.954344,volvo,good,unknown,gas,-1.259003,clean,other,fwd,unknown,sedan,red,-0.350868,-0.874508,-0.786405,-0.159058,-0.478286,-0.589782,-0.475379
426877,34990,-0.954344,cadillac,good,unknown,diesel,-1.383762,clean,other,unknown,unknown,hatchback,white,-0.350868,-0.886683,-0.794640,-0.159058,-0.479052,-0.590657,-0.475770
426878,28990,-0.729725,lexus,good,6 cylinders,gas,-0.971795,clean,other,fwd,unknown,sedan,silver,-0.328167,-0.799812,-0.737102,-0.157610,-0.467718,-0.578309,-0.469377


In [39]:
encoded_vehicles = pd.get_dummies(vehicles_cleaned8)

In [40]:
encoded_vehicles

,price,age,odometer,age^2,age odometer,odometer^2,age^3,age^2 odometer,age odometer^2,odometer^3,...,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_unknown,paint_color_white,paint_color_yellow
27,33590,-0.280488,-0.530080,-0.237366,-0.534058,-0.578695,-0.146025,-0.389122,-0.498513,-0.430164,...,False,False,False,False,False,False,False,False,True,False
28,22590,0.168749,-0.318745,-0.086030,-0.230775,-0.467509,-0.114581,-0.229509,-0.381493,-0.390946,...,False,False,False,False,False,False,False,False,False,False
29,39590,-0.954344,-1.145743,-0.350868,-0.863456,-0.772015,-0.159058,-0.477591,-0.588253,-0.474136,...,False,False,False,False,False,True,False,False,False,False
30,30990,-0.617416,-0.796895,-0.311142,-0.733809,-0.686348,-0.156032,-0.454405,-0.561710,-0.459460,...,False,False,False,False,False,True,False,False,False,False
31,15000,-0.168179,0.582934,-0.205207,-0.000423,0.264272,-0.140414,-0.226803,-0.083899,0.016553,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,-0.842034,-0.938219,-0.341409,-0.818232,-0.728575,-0.158566,-0.472391,-0.580067,-0.467930,...,False,False,False,False,False,False,False,True,False,False
426876,30590,-0.954344,-1.259003,-0.350868,-0.874508,-0.786405,-0.159058,-0.478286,-0.589782,-0.475379,...,False,False,False,False,False,True,False,False,False,False
426877,34990,-0.954344,-1.383762,-0.350868,-0.886683,-0.794640,-0.159058,-0.479052,-0.590657,-0.475770,...,False,False,False,False,False,False,False,False,True,False
426878,28990,-0.729725,-0.971795,-0.328167,-0.799812,-0.737102,-0.157610,-0.467718,-0.578309,-0.469377,...,False,False,False,False,False,False,True,False,False,False


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
ridge_model_finder = GridSearchCV(

In [ ]:
lasso_model_finder = GridSearchCV

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.